In [28]:
import pandas as pd
import os
from utils.plotter import PlotResults
from run import *

In [29]:
argv = """
--is_training --use_gpu 
--result_path scratch
--data_path Top_20.csv 
--model Transformer --scale
""".split()
parser = get_parser()
args = parser.parse_args(argv)

In [30]:
set_random_seed(args.seed)
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]
    
args.n_features = len(set(DataConfig.static_reals+DataConfig.observed_reals+DataConfig.targets))
args.n_targets = len(DataConfig.targets)

args.enc_in = args.n_features
# args.dec_in = args.c_out = args.n_targets = len(DataConfig.targets)

args.mode = 2
if args.mode == 1:
    args.dec_in = args.c_out = args.n_targets = len(DataConfig.targets)
else:
    args.dec_in = args.c_out = args.n_features
    args.n_targets = len(DataConfig.targets)

In [31]:
setting = stringify_setting(args)
exp = Exp_Forecast(args, setting)

plotter = PlotResults(
    exp.output_folder, DataConfig.targets
)

Use GPU: cuda:0

Train samples 12740, validation samples 560, test samples 560
637 days of training, 14 days of validation data, 14 days of test data.

Fitting scalers on train data


In [32]:
flag = 'val'

if flag == 'train': df = exp.train_data
elif flag == 'val': df = exp.val_data
elif flag == 'test': df = exp.test_data
else: raise NotImplementedError

preds, trues, predictions_index = exp.pred(load_model=True, flag=flag, return_index=True)
predictions_index[exp.age_data.time_index] += args.pred_len + df[exp.age_data.time_index].min()

Time encoded columns : ['month', 'day', 'weekday']
Getting valid sampling locations.
loading best model from scratch\Transformer_Top_20_scale_True_mode_2\checkpoint.pth
Preds and Trues shape: (20, 14, 1) (20, 14, 1)


In [33]:
from utils.utils import align_predictions

pred_list = [
    preds[:, :, target] for target in range(preds.shape[-1])
]

merged = align_predictions(
    df, predictions_index, 
    pred_list, exp.age_data
)
merged.head(3)

,Date,FIPS,TimeFromStart,Cases,Predicted_Cases
0,2021-11-28,2261,637,1.0,0.0
1,2021-11-29,2261,638,2.0,0.0
2,2021-11-30,2261,639,1.0,0.0


In [34]:
merged.describe()

,FIPS,TimeFromStart,Cases,Predicted_Cases
count,280.000000,280.000000,280.000000,280.000000
mean,22995.050000,643.500000,628.057143,721.968994
std,18281.790462,4.038347,998.972938,577.650269
min,2261.000000,637.000000,0.000000,0.000000
25%,6069.500000,640.000000,0.000000,501.858711
50%,14558.500000,643.500000,244.500000,628.400208
75%,39068.000000,647.000000,738.000000,709.880127
max,53033.000000,650.000000,8176.000000,3929.081543


In [35]:
_ = plotter.summed_plot(merged, type=flag)

Target Cases: MAE 590.92, RMSE 882.35, RMSLE 3.4187, NNSE 0.56368.

